# 1. 크롤링

In [146]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
import pickle

In [8]:
service = Service('C:/Users/khb16/Desktop/code/PROJECT/Recommandation/chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [136]:
pages = [] # 크롤링 오류 확인 위해 page 수 기록
title = []
lyric = []
artist = []
genres = []
writer = []
rank = []
p = re.compile("'(\d+)'")
for page in range(9551,10001,50): # 1페이지만 추출(1,51,50), 발라드 gnrCode=GN0100, 댄스 gnrCode=GN0200
    url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D\
                =&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={}".format(page) # 댄스 페이지
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 한 페이지에 있는 50개의 songId 추출
    songid = [] # songid

    for i in range(50):
        data = soup.find_all("a", class_= "btn button_icons type03 song_info")[i].get('href')
        id_ = p.search(str(data)).group(1)
        songid.append(id_)
        f = soup.select('td span.rank')[i].text
        rank.append(f)

    # 해당 songId 입력 후 정보 추출
    for song in songid:
        lyric_url = 'https://www.melon.com/song/detail.htm?songId={}'.format(song)
        driver.get(lyric_url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        a = soup.select('div.song_name')[0].text[3:].strip() # 제목
        title.append(a)
        try:
            driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
            ly = driver.find_element_by_css_selector('.lyric.on')
            b = ly.text
            lyric.append(b)
        except:
            lyric.append('가사없음')
        ar = driver.find_element_by_css_selector(".artist")
        c = ar.text
        artist.append(c)
        d = soup.select('dl.list dd')[2].text # 장르
        genres.append(d)
        try:
            e = soup.select('div.section_prdcr a.artist_name')[0].text # 작사가 1명
            writer.append(e)
        except:
            writer.append('작사가없음')
        pages.append(page)

<ipython-input-136-889516fe7638>:38: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
<ipython-input-136-889516fe7638>:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ly = driver.find_element_by_css_selector('.lyric.on')
<ipython-input-136-889516fe7638>:44: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ar = driver.find_element_by_css_selector(".artist")


IndexError: list index out of range

In [139]:
col = ['pages','rank','title','lyric','artist','genres','writer']
songlist = pd.DataFrame([pages, rank,title, lyric, artist, genres, writer]).T
songlist.columns = col
songlist

,pages,rank,title,lyric,artist,genres,writer
0,9201,9201,"Trouble (Prod. by 용준형, 김태주)",왜 종일 신경이 네게 꽂혀있는지\n나 요즘 이상해 뭐에 홀린 것 같이\n스쳐 지나갔...,크리샤 츄 (Kriesha Chu),댄스,Good Life
1,9201,9202,파라다이스 (Limited Ver.),Daylight 너무나 눈부셔\n두 발 어디론가 내딛어\nToday 떠나기 좋은 날...,DIA (다이아),댄스,STAINBOYS
2,9201,9203,허망로맨스,너와 함께 하던 순간\n나는 점점 사랑을 느꼈어\n우리 사이는 어느 순간 친구 이상...,쵸쵸우,댄스,뽕필학생
3,9201,9204,24/7,매일매일 들리는 투정들\n일주일을 반복해도 끝이 없는걸\nEveryday yeah\...,TWICE (트와이스),댄스,나연
4,9201,9205,Magic Dream,콩닥콩닥 내 심장이 떨려오는 걸\n부끄럽게 얼굴이 빨개\n자꾸만 빨개져\n어머 어머...,에이프릴 (APRIL),"댄스, 국내드라마",한준
...,...,...,...,...,...,...,...
395,None,9596,None,None,None,None,None
396,None,9597,None,None,None,None,None
397,None,9598,None,None,None,None,None
398,None,9599,None,None,None,None,None


In [142]:
songlist = songlist.head(350)
songlist

,pages,rank,title,lyric,artist,genres,writer
0,9201,9201,"Trouble (Prod. by 용준형, 김태주)",왜 종일 신경이 네게 꽂혀있는지\n나 요즘 이상해 뭐에 홀린 것 같이\n스쳐 지나갔...,크리샤 츄 (Kriesha Chu),댄스,Good Life
1,9201,9202,파라다이스 (Limited Ver.),Daylight 너무나 눈부셔\n두 발 어디론가 내딛어\nToday 떠나기 좋은 날...,DIA (다이아),댄스,STAINBOYS
2,9201,9203,허망로맨스,너와 함께 하던 순간\n나는 점점 사랑을 느꼈어\n우리 사이는 어느 순간 친구 이상...,쵸쵸우,댄스,뽕필학생
3,9201,9204,24/7,매일매일 들리는 투정들\n일주일을 반복해도 끝이 없는걸\nEveryday yeah\...,TWICE (트와이스),댄스,나연
4,9201,9205,Magic Dream,콩닥콩닥 내 심장이 떨려오는 걸\n부끄럽게 얼굴이 빨개\n자꾸만 빨개져\n어머 어머...,에이프릴 (APRIL),"댄스, 국내드라마",한준
...,...,...,...,...,...,...,...
345,9501,9546,안녕이라고 말하지마,소리내지마 우리사랑이 날아가버려\n움직이지마 우리사랑이 약해지잖아\n얘기하지마 우리...,이승철,댄스,박광현
346,9501,9547,Dangerous,아슬아슬한 보기도 딱한\n\n전화 걸긴 걸긴 좀 늦은 새벽 1시 반\n뭐라 해도 안...,f(x),댄스,Nermin Harambasic
347,9501,9548,가면무도회,Money 돈은 네게 뭐니\n만족 없는 Honey\n결국 네 모습이 싫어\nLove...,라니아,댄스,작사가없음
348,9501,9549,Yayaya (Remix Ver.),Let me seeya\nLa La La La\nLove me hey\nYa Ya ...,티아라,댄스,이트라이브 (E-TRIBE)


In [144]:
songlist = songlist.tail(300)

In [145]:
songlist.to_pickle('dance_9251-9550.pkl', protocol = 4) # 발라드는 ballad로 저장

In [219]:
songlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   pages   100 non-null    object
 1   title   100 non-null    object
 2   lyric   100 non-null    object
 3   artist  100 non-null    object
 4   genres  100 non-null    object
 5   writer  100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [ ]:
col = ['pages','rank','title','lyric','artist','genres','writer']
songlist = pd.DataFrame([pages, rank,title, lyric, artist, genres, writer]).T
songlist.columns = col
songlist

In [158]:
data1 = pd.read_pickle('dance_10001-14350.pkl')
data2 = pd.read_pickle('dance_14351-14650.pkl')
data3 = pd.read_pickle('dance_14651-20150.pkl')

In [152]:
df = pd.concat([data1, data2,data3],ignore_index=True)
#df = df.drop_duplicates()
df.to_pickle('dance_10001-20150.pkl', protocol = 4)

In [163]:
dup = []
for i in range(len(data1)):
    if data1['rank'][i] == '1':
        dup.append(i)
dup

[1250, 3100, 3400]

In [164]:
dup = []
for i in range(len(data3)):
    if data3['rank'][i] == '1':
        dup.append(i)
dup

[1950, 2600, 3600, 3750]

In [156]:
data1.head(1251)

,pages,rank,title,lyric,artist,genres,writer
0,10001,10001,Jackpot,Your turn 니가 쥔 패를 다 꺼내\n결말을 모르는\n위험해 frame out...,유호연(느와르),"댄스, 국내드라마",오현선 (lalala studio)
1,10001,10002,Gaming Christmas (Feat. 윤새),I'll be gaming on christmas\nRest in my sweet ...,정튠 (Jungtune),댄스,정튠 (Jungtune)
2,10001,10003,빛으로:We Rise,오랜 기다림\n끝이 없는 듯해\n내일은 또 오는데 oh oh\n빛과 어둠이\n오늘도...,코튼캔디 (Cotton Candy),"댄스, 국내드라마",June One
3,10001,10004,"이모와 아기새 - Bird flying (Feat. 시조새, 정수연)",Don't let me down\n일어나 늘 당당하게\nAlways care abo...,시조새의 문학 가요제,댄스,배영근
4,10001,10005,"PostCode (군대간남자 유학간여자) (Vocal. 배신행, UJ)",결국 그 날이 왔어\n그저 아무 생각 하지 말고\n별일 아닌 척 쿨한 척 하며 웃자...,PoCo,댄스,PoCo
...,...,...,...,...,...,...,...
1246,11201,11247,Feel You,I can feel you\n니 맘 알고 있어\nFeel you\n나를 떠나도 돼\...,유채영,댄스,작사가없음
1247,11201,11248,야야야두산,야야야야야야야 두산 야야야야야야야 두산\n\n야야야야 야야야야 야야야야야야 두산\n...,Various Artists,댄스,정의송
1248,11201,11249,"바보처럼 (Feat. 이시영, PS JUN)",Do you wanna love love love\n오늘 밤엔 제발 날 떠나지 말아...,전진,댄스,작사가없음
1249,11201,11250,검은고양이 네로 (Babybooster Remix Club Extended) (Fe...,가사없음,터보트로닉 (Turbotronic),댄스,작사가없음
